# 实践实验：用于手写数字识别的神经网络，多分类

在本练习中，您将使用神经网络来识别手写数字 0-9。


# 大纲
- [ 1 - 包 ](#1)
- [ 2 - ReLU 激活](#2)
- [ 3 - Softmax 函数](#3)
  - [ 练习 1](#ex01)
- [ 4 - 神经网络](#4)
  - [ 4.1 问题陈述](#4.1)
  - [ 4.2 数据集](#4.2)
  - [ 4.3 模型表示](#4.3)
  - [ 4.4 Tensorflow 模型实现](#4.4)
  - [ 4.5 Softmax 位置](#4.5)
    - [ 练习 2](#ex02)


<a name="1"></a>
## 1 - 包 

首先，让我们运行下面的单元格，导入本次作业中需要的所有包。
- [numpy](https://numpy.org/) 是 Python 科学计算的基础包。
- [matplotlib](http://matplotlib.org) 是 Python 中流行的绘图库。
- [tensorflow](https://www.tensorflow.org/) 是一个流行的机器学习平台。

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
%matplotlib widget
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

from public_tests import * 

from autils import *
from lab_utils_softmax import plt_softmax
np.set_printoptions(precision=2)

<a name="2"></a>
## 2 - ReLU 激活
本周，引入了一种新的激活函数，即修正线性单元（ReLU）。 
$$ a = max(0,z) \quad\quad\text {# ReLU 函数} $$

In [2]:
plt_act_trio()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img align="right" src="./images/C2_W2_ReLu.png"     style=" width:380px; padding: 10px 20px; " >
右侧讲座中的示例显示了 ReLU 的应用。在此示例中，派生的"意识"特征不是二元的，而是具有连续的值范围。Sigmoid 最适合开/关或二元情况。ReLU 提供连续的线性关系。此外，它有一个"关闭"范围，其中输出为零。     
"关闭"特性使 ReLU 成为非线性激活。为什么需要这个？这使多个单元能够在不干扰的情况下对结果函数做出贡献。这在支持的可选实验中有更详细的检查。 

<a name="3"></a>
## 3 - Softmax 函数
多分类神经网络生成 N 个输出。其中一个输出被选作预测答案。在输出层中，向量 $\mathbf{z}$ 由一个线性函数生成，该函数应用于 softmax 函数。softmax 函数将 $\mathbf{z}$ 转换为如下所述的概率分布。应用 softmax 后，每个输出将在 0 到 1 之间，并且输出的总和为 1，因此它们可以被解释为概率。较大的输入将对应于较大的输出概率。
<center>  <img  src="./images/C2_W2_NNSoftmax.PNG" width="600" />  

softmax 函数可以写成：
$$a_j = \frac{e^{z_j}}{ \sum_{k=0}^{N-1}{e^{z_k} }} \tag{1}$$

其中 $z = \mathbf{w} \cdot \mathbf{x} + b$，N 是输出层中特征/类别的数量。  

<a name="ex01"></a>
### 练习 1
让我们创建一个 NumPy 实现：

In [7]:
# UNQ_C1
# GRADED CELL: my_softmax

def my_softmax(z):  
    """ Softmax converts a vector of values to a probability distribution.
    Args:
      z (ndarray (N,))  : input data, N features
    Returns:
      a (ndarray (N,))  : softmax of z
    """    
    ### START CODE HERE ### 
    ez = np.exp(z)
    a = ez/np.sum(ez)
    ### END CODE HERE ### 
    return a

In [8]:
z = np.array([1., 2., 3., 4.])
a = my_softmax(z)
atf = tf.nn.softmax(z)
print(f"my_softmax(z):         {a}")
print(f"tensorflow softmax(z): {atf}")

# BEGIN UNIT TEST  
test_my_softmax(my_softmax)
# END UNIT TEST  

my_softmax(z):         [0.03 0.09 0.24 0.64]
tensorflow softmax(z): [0.03 0.09 0.24 0.64]
 All tests passed.


<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    One implementation uses for loop to first build the denominator and then a second loop to calculate each output.
    
```python
def my_softmax(z):  
    N = len(z)
    a =                     # initialize a to zeros 
    ez_sum =                # initialize sum to zero
    for k in range(N):      # loop over number of outputs             
        ez_sum +=           # sum exp(z[k]) to build the shared denominator      
    for j in range(N):      # loop over number of outputs again                
        a[j] =              # divide each the exp of each output by the denominator   
    return(a)
```
<details>
  <summary><font size="3" color="darkgreen"><b>Click for code</b></font></summary>
   
```python
def my_softmax(z):  
    N = len(z)
    a = np.zeros(N)
    ez_sum = 0
    for k in range(N):                
        ez_sum += np.exp(z[k])       
    for j in range(N):                
        a[j] = np.exp(z[j])/ez_sum   
    return(a)

Or, a vector implementation:

def my_softmax(z):  
    ez = np.exp(z)              
    a = ez/np.sum(ez)           
    return(a)

```


下面，改变 `z` 输入的值。特别注意分子中的指数如何放大值之间的微小差异。还要注意输出值的总和为一。

In [9]:
plt.close("all")
plt_softmax(my_softmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<a name="4"></a>
## 4 - Neural Networks

In last weeks assignment, you implemented a neural network to do binary classification. This week you will extend that to multiclass classification. This will utilize the softmax activation.


<a name="4.1"></a>
### 4.1 Problem Statement

In this exercise, you will use a neural network to recognize ten handwritten digits, 0-9. This is a multiclass classification task where one of n choices is selected. Automated handwritten digit recognition is widely used today - from recognizing zip codes (postal codes) on mail envelopes to recognizing amounts written on bank checks. 


<a name="4.2"></a>
### 4.2 Dataset

You will start by loading the dataset for this task. 
- The `load_data()` function shown below loads the data into variables `X` and `y`


- The data set contains 5000 training examples of handwritten digits $^1$.  

    - Each training example is a 20-pixel x 20-pixel grayscale image of the digit. 
        - Each pixel is represented by a floating-point number indicating the grayscale intensity at that location. 
        - The 20 by 20 grid of pixels is “unrolled” into a 400-dimensional vector. 
        - Each training examples becomes a single row in our data matrix `X`. 
        - This gives us a 5000 x 400 matrix `X` where every row is a training example of a handwritten digit image.

$$X = 
\left(\begin{array}{cc} 
--- (x^{(1)}) --- \\
--- (x^{(2)}) --- \\
\vdots \\ 
--- (x^{(m)}) --- 
\end{array}\right)$$ 

- The second part of the training set is a 5000 x 1 dimensional vector `y` that contains labels for the training set
    - `y = 0` if the image is of the digit `0`, `y = 4` if the image is of the digit `4` and so on.

$^1$<sub> This is a subset of the MNIST handwritten digit dataset (http://yann.lecun.com/exdb/mnist/)</sub>

In [10]:
# load dataset
X, y = load_data()

#### 4.2.1 查看变量
让我们更熟悉您的数据集。  
- 一个好的起点是打印出每个变量并查看它包含的内容。

下面的代码打印变量 `X` 和 `y` 中的第一个元素。  

In [ ]:
print ('The first element of X is: ', X[0])

In [11]:
print ('The first element of y is: ', y[0,0])
print ('The last element of y is: ', y[-1,0])

The first element of y is:  0
The last element of y is:  9


#### 4.2.2 检查变量的维度

另一种熟悉数据的方法是查看其维度。请打印 `X` 和 `y` 的形状，并查看数据集中有多少训练样本。

In [12]:
print ('The shape of X is: ' + str(X.shape))
print ('The shape of y is: ' + str(y.shape))

The shape of X is: (5000, 400)
The shape of y is: (5000, 1)


#### 4.2.3 可视化数据

您将从可视化训练集的子集开始。 
- 在下面的单元格中，代码从 `X` 中随机选择 64 行，将每一行映射回 20 像素 x 20 像素的灰度图像，并一起显示这些图像。 
- 每个图像的标签显示在图像上方 

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# You do not need to modify anything in this cell

m, n = X.shape

fig, axes = plt.subplots(8,8, figsize=(5,5))
fig.tight_layout(pad=0.13,rect=[0, 0.03, 1, 0.91]) #[left, bottom, right, top]

#fig.tight_layout(pad=0.5)
widgvis(fig)
for i,ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)
    
    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = X[random_index].reshape((20,20)).T
    
    # Display the image
    ax.imshow(X_random_reshaped, cmap='gray')
    
    # Display the label above the image
    ax.set_title(y[random_index,0])
    ax.set_axis_off()
    fig.suptitle("Label, image", fontsize=14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<a name="4.3"></a>
### 4.3 模型表示

您将在本次作业中使用的神经网络如下图所示。 
- 这有两个带 ReLU 激活的密集层，后跟一个带线性激活的输出层。 
    - 回想一下，我们的输入是数字图像的像素值。
    - 由于图像大小为 $20\times20$，这给了我们 $400$ 个输入  
    
<img src="images/C2_W2_Assigment_NN.png" width="600" height="450">

- 参数的维度是为具有第 1 层 $25$ 个单元、第 2 层 $15$ 个单元和第 3 层 $10$ 个输出单元（每个数字一个）的神经网络调整大小的。

    - 回想一下，这些参数的维度确定如下：
        - 如果网络在一层中有 $s_{in}$ 个单元，在下一层中有 $s_{out}$ 个单元，那么 
            - $W$ 的维度将是 $s_{in} \times s_{out}$。
            - $b$ 将是一个具有 $s_{out}$ 个元素的向量
  
    - 因此，`W` 和 `b` 的形状是 
        - layer1: `W1` 的形状是 (400, 25)，`b1` 的形状是 (25,)
        - layer2: `W2` 的形状是 (25, 15)，`b2` 的形状是: (15,)
        - layer3: `W3` 的形状是 (15, 10)，`b3` 的形状是: (10,)
>**注意：** 偏置向量 `b` 可以表示为 1-D (n,) 或 2-D (n,1) 数组。Tensorflow 使用 1-D 表示，本实验将保持该约定： 
               

<a name="4.4"></a>
### 4.4 Tensorflow 模型实现


Tensorflow 模型是逐层构建的。层的输入维度（上面的 $s_{in}$）是为您计算的。您指定层的*输出维度*，这决定了下一层的输入维度。第一层的输入维度是从下面 `model.fit` 语句中指定的输入数据的大小推导出来的。 
>**注意：** 也可以添加一个指定第一层输入维度的输入层。例如：  
`tf.keras.Input(shape=(400,)),    #指定输入形状`  
我们将在这里包含它来说明一些模型大小。

<a name="4.5"></a>
### 4.5 Softmax 位置
如讲座和可选的 softmax 实验所述，如果在训练期间将 softmax 与损失函数分组而不是与输出层分组，则数值稳定性会提高。这在*构建*模型和*使用*模型时都有影响。  
构建：  
* 最终的 Dense 层应使用 'linear' 激活。这实际上是没有激活。 
* `model.compile` 语句将通过包含 `from_logits=True` 来指示这一点。
`loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) `  
* 这不会影响目标的形式。在 SparseCategorialCrossentropy 的情况下，目标是期望的数字，0-9。

使用模型：
* 输出不是概率。如果需要输出概率，请应用 softmax 函数。

<a name="ex02"></a>
### 练习 2

下面，使用 Keras [Sequential 模型](https://keras.io/guides/sequential_model/)和带 ReLU 激活的 [Dense 层](https://keras.io/api/layers/core_layers/dense/)来构建上面描述的三层网络。

In [16]:
# UNQ_C2
# GRADED CELL: Sequential model
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        ### START CODE HERE ### 
        tf.keras.layers.InputLayer((400,)),
        tf.keras.layers.Dense(25, activation="relu", name="L1"),
        tf.keras.layers.Dense(15, activation="relu", name="L2"),
        tf.keras.layers.Dense(10, activation="linear", name="L3")
        ### END CODE HERE ### 
    ], name = "my_model" 
)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [17]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 25)                10025     
                                                                 
 L2 (Dense)                  (None, 15)                390       
                                                                 
 L3 (Dense)                  (None, 10)                160       
                                                                 
Total params: 10,575
Trainable params: 10,575
Non-trainable params: 0
_________________________________________________________________


<details>
  <summary><font size="3" color="darkgreen"><b>Expected Output (Click to expand)</b></font></summary>
The `model.summary()` function displays a useful summary of the model. Note, the names of the layers may vary as they are auto-generated unless the name is specified.    
    
```
Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
L1 (Dense)                   (None, 25)                10025     
_________________________________________________________________
L2 (Dense)                   (None, 15)                390       
_________________________________________________________________
L3 (Dense)                   (None, 10)                160       
=================================================================
Total params: 10,575
Trainable params: 10,575
Non-trainable params: 0
_________________________________________________________________
```

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
```python
tf.random.set_seed(1234)
model = Sequential(
    [               
        ### START CODE HERE ### 
        tf.keras.Input(shape=(400,)),     # @REPLACE 
        Dense(25, activation='relu', name = "L1"), # @REPLACE 
        Dense(15, activation='relu',  name = "L2"), # @REPLACE  
        Dense(10, activation='linear', name = "L3"),  # @REPLACE 
        ### END CODE HERE ### 
    ], name = "my_model" 
)
``` 

In [18]:
# BEGIN UNIT TEST     
test_model(model, 10, 400)
# END UNIT TEST     

All tests passed!


摘要中显示的参数计数对应于权重和偏置数组中的元素数量，如下所示。

让我们进一步检查权重，以验证 tensorflow 产生的维度与我们上面计算的相同。

In [19]:
[layer1, layer2, layer3] = model.layers

In [ ]:
#### 检查权重形状
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

W1 shape = (400, 25), b1 shape = (25,)
W2 shape = (25, 15), b2 shape = (15,)
W3 shape = (15, 10), b3 shape = (10,)


**Expected Output**
```
W1 shape = (400, 25), b1 shape = (25,)  
W2 shape = (25, 15), b2 shape = (15,)  
W3 shape = (15, 10), b3 shape = (10,)
```

以下代码：
* 定义一个损失函数 `SparseCategoricalCrossentropy`，并通过添加 `from_logits=True` 来指示 softmax 应包含在损失计算中）
* 定义一个优化器。一个流行的选择是自适应矩（Adam），这在讲座中已描述。

In [21]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

history = model.fit(
    X,y,
    epochs=40
)

Epoch 1/40
157/157 [==============================] - 1s 2ms/step - loss: 1.7094
Epoch 2/40
157/157 [==============================] - 0s 2ms/step - loss: 0.7480
Epoch 3/40
157/157 [==============================] - 0s 2ms/step - loss: 0.4428
Epoch 4/40
157/157 [==============================] - 0s 2ms/step - loss: 0.3463
Epoch 5/40
157/157 [==============================] - 0s 2ms/step - loss: 0.2977
Epoch 6/40
157/157 [==============================] - 0s 2ms/step - loss: 0.2630
Epoch 7/40
157/157 [==============================] - 0s 2ms/step - loss: 0.2361
Epoch 8/40
157/157 [==============================] - 0s 2ms/step - loss: 0.2131
Epoch 9/40
157/157 [==============================] - 0s 2ms/step - loss: 0.2004
Epoch 10/40
157/157 [==============================] - 0s 2ms/step - loss: 0.1805
Epoch 11/40
157/157 [==============================] - 0s 2ms/step - loss: 0.1692
Epoch 12/40
157/157 [==============================] - 0s 2ms/step - loss: 0.1580
Epoch 13/40
157/157 [====

#### Epochs 和 batches
在上面的 `compile` 语句中，`epochs` 的数量设置为 100。这指定整个数据集应在训练期间应用 100 次。在训练期间，您会看到描述训练进度的输出，如下所示：
```
Epoch 1/100
157/157 [==============================] - 0s 1ms/step - loss: 2.2770
```
第一行 `Epoch 1/100` 描述模型当前正在运行的 epoch。为了提高效率，训练数据集被分成"批次"。Tensorflow 中批次的默认大小为 32。我们的数据集中有 5000 个样本，大约 157 个批次。第二行上的符号 `157/157 [====` 描述已执行了哪个批次。

#### Loss（代价）
在课程 1 中，我们学会了通过监控代价来跟踪梯度下降的进度。理想情况下，代价会随着算法迭代次数的增加而减少。Tensorflow 将代价称为 `loss`。上面，您看到在 `model.fit` 执行时每个 epoch 显示的损失。[.fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model) 方法返回包括损失在内的各种指标。这在上面的 `history` 变量中捕获。这可用于在如下所示的图中检查损失。

In [22]:
plot_loss_tf(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 预测 
要进行预测，请使用 Keras `predict`。下面，X[1015] 包含一个数字 2 的图像。

In [23]:
image_of_two = X[1015]
display_digit(image_of_two)

prediction = model.predict(image_of_two.reshape(1,400))  # prediction

print(f" predicting a Two: \n{prediction}")
print(f" Largest Prediction index: {np.argmax(prediction)}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 predicting a Two: 
[[ -7.99  -2.23   0.77  -2.41 -11.66 -11.15  -9.53  -3.36  -4.42  -7.17]]
 Largest Prediction index: 2


最大输出是 prediction[2]，表示预测的数字是 '2'。如果问题只需要选择，这就足够了。使用 NumPy [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 来选择它。如果问题需要概率，则需要 softmax：

In [24]:
prediction_p = tf.nn.softmax(prediction)

print(f" predicting a Two. Probability vector: \n{prediction_p}")
print(f"Total of predictions: {np.sum(prediction_p):0.3f}")

 predicting a Two. Probability vector: 
[[1.42e-04 4.49e-02 8.98e-01 3.76e-02 3.61e-06 5.97e-06 3.03e-05 1.44e-02
  5.03e-03 3.22e-04]]
Total of predictions: 1.000


要返回表示预测目标的整数，您需要最大概率的索引。这可以通过 Numpy [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函数来完成。

In [25]:
yhat = np.argmax(prediction_p)

print(f"np.argmax(prediction_p): {yhat}")

np.argmax(prediction_p): 2


让我们比较 64 个随机数字样本的预测与标签。这需要一些时间才能运行。

In [26]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# You do not need to modify anything in this cell

m, n = X.shape

fig, axes = plt.subplots(8,8, figsize=(5,5))
fig.tight_layout(pad=0.13,rect=[0, 0.03, 1, 0.91]) #[left, bottom, right, top]
widgvis(fig)
for i,ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)
    
    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = X[random_index].reshape((20,20)).T
    
    # Display the image
    ax.imshow(X_random_reshaped, cmap='gray')
    
    # Predict using the Neural Network
    prediction = model.predict(X[random_index].reshape(1,400))
    prediction_p = tf.nn.softmax(prediction)
    yhat = np.argmax(prediction_p)
    
    # Display the label above the image
    ax.set_title(f"{y[random_index,0]},{yhat}",fontsize=10)
    ax.set_axis_off()
fig.suptitle("Label, yhat", fontsize=14)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

让我们看看一些错误。 
>注意：增加训练 epoch 的数量可以消除此数据集上的错误。

In [27]:
print( f"{display_errors(model,X,y)} errors out of {len(X)} images")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

15 errors out of 5000 images


### 恭喜！
您已经成功构建并利用神经网络进行多分类。